## Load libraries

In [1]:
import numpy as np
from tqdm import tqdm
import time
from torch.utils.tensorboard import SummaryWriter
from multiprocessing.pool import ThreadPool
from datetime import datetime
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
from torch.distributions.categorical import Categorical
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.envs.unity_gym_env import UnityToGymWrapper

## Parameters

In [2]:
device = (torch.device("cuda"))
dim = 256
critic_lr = 3e-4
actor_lr = critic_lr / 3.0
reparam_noise = 1e-6
gamma=0.99
tau=0.005
alpha_start = 1
max_size = 1000000
batch_size = 512
total_plays = 50000
num_epochs = 1
N = 1

## Environment setup

In [3]:
unity_env = UnityEnvironment("D:\Practice\SentisInfrence\Build\SentisInfrence.exe", no_graphics=True)
env = UnityToGymWrapper(unity_env)

#env = gym.make("CartPole-v1")

obs_dim = env.observation_space.shape
n_actions=env.action_space.n

print(obs_dim)
print(n_actions)

[WARNING] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
(16,)
4


In [ ]:
env.close()

## Replay buffer

In [4]:
class ReplayBuffer():
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape))
        self.new_state_memory = np.zeros((self.mem_size, *input_shape))
        self.action_memory = np.zeros((self.mem_size, n_actions))
        self.reward_memory = np.zeros(self.mem_size)
        self.done_memory = np.zeros((self.mem_size), dtype=bool)

    def store_transition(self, state, action, reward, state_, dones):
        index = self.mem_cntr % self.mem_size

        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.done_memory[index] = dones

        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)

        batch = np.random.choice(max_mem, batch_size)

        states = self.state_memory[batch]
        states_ = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        dones = self.done_memory[batch]

        return states, actions, rewards, states_, dones

## Networks

In [5]:
class CriticNetwork(nn.Module):
    def __init__(self):
        super(CriticNetwork, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(obs_dim[-1], dim),
            nn.ReLU(),
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, n_actions)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=critic_lr)
        self.scheduler = optim.lr_scheduler.LinearLR(self.optimizer, start_factor=1, end_factor=0, total_iters=total_plays)

        self.to(device)

    def forward(self, state):
        q = self.layers(state)

        return q

class ActorNetwork(nn.Module):
    def __init__(self):
        super(ActorNetwork, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(*obs_dim, dim),
            nn.ReLU(),
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, int(n_actions)),
            nn.Softmax(dim=-1)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=actor_lr)
        self.scheduler = optim.lr_scheduler.LinearLR(self.optimizer, start_factor=1, end_factor=0, total_iters=total_plays)

        self.to(device)

    def forward(self, state):
        action_probs = self.layers(state)
       

        return action_probs

    def sample_normal(self, state):
        action_probs = self.forward(state)
        
        dist = Categorical(action_probs)
        action = dist.sample()

        z = action_probs == 0.0
        z = z.float() * 1e-8
        log_probs = torch.log(action_probs + z)

        return action, log_probs, action_probs
    
class Agent():
    def __init__(self):
        self.memory = ReplayBuffer(max_size, obs_dim, n_actions)

        self.actor = ActorNetwork()
        self.critic_1 = CriticNetwork()
        self.critic_2 = CriticNetwork()
        self.critic_1_target = CriticNetwork()
        self.critic_2_target = CriticNetwork()

        self.critic_1_target.load_state_dict(self.critic_1.state_dict())
        self.critic_2_target.load_state_dict(self.critic_2.state_dict())

        self.alpha = alpha_start
        self.log_alpha = torch.zeros(1, requires_grad=True, device=device)
        self.alpha_optimizer = torch.optim.Adam([self.log_alpha], lr=critic_lr)
        self.alpha_scheduler = optim.lr_scheduler.LinearLR(self.alpha_optimizer, start_factor=1, end_factor=0, total_iters=total_plays)
        self.target_entropy = -n_actions

    def choose_action(self, observation):
        state = torch.tensor(np.array([observation], dtype=float), dtype=torch.float).to(device)
        actions, _, _ = self.actor.sample_normal(state)

        return actions.cpu().detach().numpy()[0]

    def remember(self, state, action, reward, new_state, dones):
        self.memory.store_transition(state, action, reward, new_state, dones)

    def gradient_step(self):
        if self.memory.mem_cntr < batch_size:
            return
        
        for _ in range(num_epochs):
            state, actions, reward, state_, dones = self.memory.sample_buffer(batch_size)

            reward = torch.tensor(reward, dtype=torch.float).to(device)
            state_ = torch.tensor(state_, dtype=torch.float).to(device)
            state = torch.tensor(state, dtype=torch.float).to(device)
            actions = torch.tensor(actions, dtype=torch.float).to(device)
            dones = torch.tensor(dones, dtype=torch.bool).to(device)

            # Critics gradient step
            _, _, action_probs_ = self.actor.sample_normal(state_)
            
            with torch.no_grad():
                q1_target_value = self.critic_1_target.forward(state_)
                q2_target_value = self.critic_2_target.forward(state_)
                q_target_value = torch.min(q2_target_value, q1_target_value) * action_probs_
                q_hat = reward.view(batch_size, -1) + gamma * ~(dones.view(batch_size, -1)) * q_target_value
            q1_value = self.critic_1.forward(state).gather(1, actions.long())
            q2_value = self.critic_2.forward(state).gather(1, actions.long())
            q1_loss = 0.5 * F.mse_loss(q1_value, q_hat)
            q2_loss = 0.5 * F.mse_loss(q2_value, q_hat)
            
            q_loss = q1_loss + q2_loss
            self.critic_1.zero_grad()
            self.critic_2.zero_grad()
            q_loss.backward()
            self.critic_1.optimizer.step()
            self.critic_2.optimizer.step()

            # Policy gradient step
            _, log_probs, action_probs = self.actor.sample_normal(state)
            q1_value = self.critic_1.forward(state)
            q2_value = self.critic_2.forward(state)
            q_value = torch.min(q1_value, q2_value)
            actor_loss = (action_probs * (self.alpha * log_probs - q_value)).mean()
            self.actor.zero_grad()
            actor_loss.backward()
            self.actor.optimizer.step()

            # Alpha gradient step
            _, log_probs, action_probs = self.actor.sample_normal(state)
            alpha_loss = -(action_probs * self.log_alpha * (log_probs + self.target_entropy).detach()).mean()
            self.alpha_optimizer.zero_grad()
            alpha_loss.backward()
            self.alpha_optimizer.step()
            self.alpha = self.log_alpha.exp().item()

            # Target critic weights update
            for param, target_param in zip(self.critic_1.parameters(), self.critic_1_target.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

            for param, target_param in zip(self.critic_2.parameters(), self.critic_2_target.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

            return
    
    def save_model(self):
        model_scripted = torch.jit.script(self.actor)
        model_scripted.save("models/unity_test" + "_final.pth")

## Training

In [6]:
agent = Agent()

In [42]:
agent.save_model()

In [14]:
if "pbar" in globals():
    pbar.close()
pbar = tqdm(total=total_plays)
pbar.reset()
writer = SummaryWriter("logs/unity_test" + str(datetime.now().day) + str(datetime.now().hour) + str(datetime.now().minute))

writer.add_text(
          "Hyperparameters",
          "|param|value|\n|-|-|\n%s" % ("\n".join(
               [f"|Critic lr|{critic_lr}|",
                f"|Actor lr|{actor_lr}|",
                f"|Layer dim|{dim}|",
                f"|Batch size|{batch_size}|",
                f"|Gamma|{gamma}|",
                f"|Tau|{tau}|",
                ]
          )),
          int(str(datetime.now().day) + str(datetime.now().hour) + str(datetime.now().minute)))

#agent = Agent()
best_score = -100000
score_history = []

global_step = 0
for i in range(total_plays):
    observation = env.reset()
    done = False
    score = 0
    iter_steps = 0
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, terminated, _ = env.step(action)
        done = terminated
        score += reward
        agent.remember(observation, action, reward, observation_, terminated)
        if iter_steps % N == 0:
            agent.gradient_step()
        iter_steps += 1
        global_step += 1
        observation = observation_
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])

    if avg_score > best_score:
        best_score = avg_score
        agent.save_model()

    if global_step > batch_size:
        agent.critic_1.scheduler.step()
        agent.critic_2.scheduler.step()
        agent.actor.scheduler.step()
        agent.alpha_scheduler.step()

    writer.add_scalar("charts/reward", avg_score, global_step=global_step)
    writer.add_scalar("charts/step_count", iter_steps, global_step=global_step)
    pbar.update()

pbar.close() 

100%|██████████| 50000/50000 [5:12:05<00:00,  2.67it/s]   


## Weights save

In [15]:
torch.save({
            'actor_state_dict': agent.actor.state_dict(),
            'critic1_state_dict': agent.critic_1.state_dict(),
            'critic2_state_dict': agent.critic_2.state_dict(),
            'target_critic1_state_dict': agent.critic_1_target.state_dict(),
            'target_critic2_state_dict': agent.critic_2_target.state_dict(),
            'actor_optimizer_state_dict': agent.actor.optimizer.state_dict(),
            'critic1_optimizer_state_dict': agent.critic_1.optimizer.state_dict(),
            'critic2_optimizer_state_dict': agent.critic_2.optimizer.state_dict(),
            }, "models/unity" + str(i) + "_steps_weights.pt")

## Load weights

In [7]:
ckpt = torch.load("models/unity49999_steps_weights.pt")
agent.actor.load_state_dict(ckpt['actor_state_dict'])
agent.actor.optimizer.load_state_dict(ckpt['actor_optimizer_state_dict'])
agent.critic_1.load_state_dict(ckpt['critic1_state_dict'])
agent.critic_2.load_state_dict(ckpt['critic2_state_dict'])
agent.critic_1.optimizer.load_state_dict(ckpt['critic1_optimizer_state_dict'])
agent.critic_2.optimizer.load_state_dict(ckpt['critic2_optimizer_state_dict'])
agent.critic_1_target.load_state_dict(ckpt['target_critic1_state_dict'])
agent.critic_2_target.load_state_dict(ckpt['target_critic2_state_dict'])
agent.actor.train()
agent.critic_1.train()
agent.critic_2.train()

CriticNetwork(
  (layers): Sequential(
    (0): Linear(in_features=16, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=4, bias=True)
  )
)

## Export to ONNX (Unity format)

In [8]:
observation = env.reset() 
observation = torch.tensor(np.array([observation]), dtype=torch.float).to(device)

class WrapperNet(torch.nn.Module):
    def __init__(self, actor):
        """
        Wraps the VisualQNetwork adding extra constants and dummy mask inputs
        required by runtime inference with Sentis.

        For environment continuous actions outputs would need to add them
        similarly to how discrete action outputs work, both in the wrapper
        and in the ONNX output_names / dynamic_axes.
        """
        super(WrapperNet, self).__init__()
        self.qnet = actor

        # version_number
        #   MLAgents1_0 = 2   (not covered by this example)
        #   MLAgents2_0 = 3
        version_number = torch.Tensor([3])
        self.version_number = nn.Parameter(version_number, requires_grad=False)

        # memory_size
        # TODO: document case where memory is not zero.
        memory_size = torch.Tensor([0])
        self.memory_size = nn.Parameter(memory_size, requires_grad=False)

        # discrete_action_output_shape
        output_shape = torch.Tensor([n_actions])
        self.discrete_shape = nn.Parameter(torch.tensor(output_shape), requires_grad=False)


    # if you have discrete actions ML-agents expects corresponding a mask
    # tensor with the same shape to exist as input
    def forward(self, obs: torch.tensor):
        probs = self.qnet(obs)
        
        action = torch.argmax(probs, dim=-1, keepdim=True)
        
        return [action], self.discrete_shape, self.version_number, self.memory_size
    
model = WrapperNet(agent.actor)

torch.onnx.export(
    model,
    observation,
    'UnityTest.onnx',
    opset_version=11,
    # input_names must correspond to the WrapperNet forward parameters
    # obs will be obs_0, obs_1, etc.
    input_names=["obs_0"],
    # output_names must correspond to the return tuple of the WrapperNet
    # forward function.
    output_names=["discrete_actions", "discrete_action_output_shape",
                  "version_number", "memory_size"],
    # All inputs and outputs should have their 0th dimension be designated
    # as 'batch'
    dynamic_axes={'obs_0': {0: 'batch'},
                  'discrete_actions': {0: 'batch'},
                  'discrete_action_output_shape': {0: 'batch'}
                 }
    )

C:\Users\Vefery\AppData\Local\Temp\ipykernel_7096\29769737.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.discrete_shape = nn.Parameter(torch.tensor(output_shape), requires_grad=False)
